<span style="font-size:2.5em;">Converts fiber capture images to rgb matrix txt files</span> \
<span style="font-size:2em;">Each fiber capture has X and Y which is then used to produce 6 cross sections each</span>

In [ ]:
import os
import numpy as np
from numpy import asarray
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import random
import re


In [ ]:
path = 'C:/Users/landy/Documents/VSCODE/ML/Capture_SN030055_20240702114908_0001_X.png'
img = Image.open(path)
print(img.size)

img_arr = asarray(img)
print(img_arr)


(512, 512)
[[105 105 106 ... 107 106 108]
 [106 107 106 ... 106 104 107]
 [106 106 105 ... 106 105 105]
 ...
 [106 104 109 ... 107 104 108]
 [106 104 104 ... 105 106 104]
 [105 104 106 ... 107 103 106]]


In [ ]:
def zoom_image(image, center_x, center_y, zoom_factor, output_size):
    # Calculate the new width and height based on the zoom factor
    new_width = int(output_size[0] / zoom_factor)
    new_height = int(output_size[1] / zoom_factor)

    # Calculate the top-left and bottom-right coordinates of the new zoomed-in section
    top_left_x = max(center_x - new_width // 2, 0)
    top_left_y = max(center_y - new_height // 2, 0)
    bottom_right_x = min(center_x + new_width // 2, image.shape[1])
    bottom_right_y = min(center_y + new_height // 2, image.shape[0])

    # Crop the image
    cropped_image = image[top_left_y:bottom_right_y, top_left_x:bottom_right_x]

    # Resize the cropped image back to the desired output size
    zoomed_image = cv2.resize(cropped_image, output_size, interpolation=cv2.INTER_LINEAR)

    return zoomed_image


In [ ]:
# Change based on fiber image path
fiber_name = 'EZ'
newpath = 'C:/Users/landy/Documents/VSCODE/fibers/' + fiber_name
if not os.path.exists(newpath):
  os.makedirs(newpath)

img_folder = 'C:/Users/landy/Documents/VSCODE/' + fiber_name + '_CAPTURES'
out_folder = newpath


# Getting unique image name
for fiber in os.listdir(img_folder):
    fiber_path = os.path.join(img_folder, fiber)
    img = Image.open(fiber_path)
    img_arr = asarray(img)
    name = fiber[0:-4] + '.txt'

    # Get 3 different 21 pixel cross sections from left fiber end
    # Starting at 50, 100, 150 pixels
    for l in range(1, 4):
        locals()["img_arrL" + str(l)] = img_arr[180:331, l * 50 : l * 50 + 21]
        with open(os.path.join(out_folder, 'L' + str(l) + '_' + name), 'w') as file:
            for line in locals()["img_arrL" + str(l)].astype(str):
                file.write(",".join(line) + "\n")

    # Get 3 different 21 pixel cross sections from right fiber end
    # Starting at 512 - 150 - 21, 512 - 100 - 21, 512 - 50 - 21 pixels
    for r in range(1, 4):
        locals()["img_arrR" + str(r)] = img_arr[180:331, 512 - r * 50 - 21: 512 - r * 50]
        with open(os.path.join(out_folder, 'R' + str(r) + '_' + name), 'w') as file:
            for line in locals()["img_arrR" + str(r)].astype(str):
                file.write(",".join(line) + "\n")


In [ ]:
# Testing
# Function to read the text file and convert it to a 2D numpy array of integers
def textfile_to_numpy_array(filename):
    matrix = []
    with open(filename, 'r') as file:
        for line in file.readlines():
            try:
                # Convert line to a list of integers
                row = [int(num) for num in line.split(',')]
                matrix.append(row)

            except ValueError:
                # If any line cannot be converted to integers, skip the file
                continue

    # Convert to a numpy array
    matrix = np.array(matrix).flatten()
    print(matrix)
    return matrix



In [ ]:
bit_path = 'C:/Users/landy/Documents/VSCODE/EZ_CONVERT/L1_00426462A1_3_X.txt'
#bitmap = textfile_to_numpy_array(bit_path)
def zoom_image(bitmap, zoom_factor):
    bitmap = bitmap.reshape(151, 21)
    height = bitmap.shape[0]
    width = bitmap.shape[1]

    pil_image = Image.fromarray(np.uint8(bitmap))  # Convert to PIL image and scale pixel values
    new_width, new_height = int(width * zoom_factor), int(height * zoom_factor)
    pil_image = pil_image.resize((new_width, new_height), Image.NEAREST)  # Use NEAREST to avoid interpolation

    return np.array(pil_image)  # Convert back to numpy array and normalize pixel values

In [ ]:

# new_img = zoom_image(textfile_to_numpy_array(bit_path), 1.5)[35:186, 5:26]
# [75:226, 10:31]
# [35:186, 5:26]
# with open(bit_path, 'w') as file:
#     file.truncate(0)
#     for line in new_img:
#         line_str = [str(n) for n in line] # create string list from integer list
#         file.write(', '.join(line_str) + '\n')

# plt.imshow(textfile_to_numpy_array(bit_path).reshape(151,21))

In [ ]:
for bitmaps in os.listdir(out_folder):
    zoom_factor = random.choice([1, 1.5, 2])
    if zoom_factor == 1:
        continue
    if zoom_factor == 2:
        new_img = zoom_image(textfile_to_numpy_array(bit_path), zoom_factor)[75:228, 10:31]
    if zoom_factor == 1.5:
        new_img = zoom_image(textfile_to_numpy_array(bit_path), zoom_factor)[35:188, 5:26]

    with open(os.path.join(out_folder, bitmaps), 'r+') as file:
        file.truncate(0)
        for line in new_img:
            line_str = [str(n) for n in line]
            file.write(','.join(line_str) + '\n')

        file.close

[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ... 7 7 8]
[8 8 8 ...

In [ ]:
# Example usage
convert_path = 'C:/Users/landy/Documents/VSCODE/EZ_CONVERT'
file_list = os.listdir(convert_path)
random_file = random.choice(file_list)

rand_arr = textfile_to_numpy_array(os.path.join(convert_path, random_file))
rand_arr = rand_arr.reshape(151, 21)
print(random_file)
plt.imshow(rand_arr)

In [ ]:
len(file_list)

1056